In [1]:
### This script computes the translational diffusion coefficients of a crowded protein trajectory: blockwise, per protein

### Inputs: TPR file, XTC files, protein_indeces.txt files, and Proteome Class code
### Output: DT diffusion coefficients

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../ProteomeClass/'))
from proteome import *
import re
sys.path.append('source_codes/') 
from DC_correction_PBC import *


In [ ]:
#### USER modifications #####################################################################

base         = '../../../Data.Availability/Upload/TRJ_Psychro_sys4/System4/'
directory    = base + 'Folded/'
indeces_file = base + 'Files/protein_indeces.txt' 
TPR_file     = directory + '300K/sys.tpr'
XTC_file     = '/traj_prot.xtc'

output_dir   = 'Results'

#############################################################################################

pattern      = re.compile(r'^\d+K$')
temp_dirs    = [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name)) and pattern.match(name)]
temps        = [int(t[:-1]) for t in temp_dirs]

# Create Proteome Object
prot1 = Proteome(tpr_file     = TPR_file, 
                 indices_path = indeces_file, 
                 outputdir    = output_dir , 
                 base         = 'P_Arcticus',
                 plot         = True,
                 selection    = "protein"
                )

# Loop over the temperatures to find the blockwise translational diffusion
DT = []
DT_err = []

for T_dir in temp_dirs:
    print('Processing Temperature: ', T_dir)
    prot1.add_trajectory(directory + T_dir + XTC_file)
    prot1.center_of_masses(append=False)
    prot1.msd_com(n_blocks=5)
    prot1.msd_com_weighted()
    dt = prot1.dt_com_weighted()
    DT.append(dt)
    DT_err.append(prot1.D_wavg_err)

DT = np.array(DT)
DT_err = np.array(DT_err)

### Correct DT

In [22]:
visc_array = '../../../Data.Availability/Upload/TRJ_Psychro_sys4/System4/Examples/Viscosity/Psychro/fit_BDtDE.npy' # Array of viscosity parameters
DT_corr = DT + get_correction_crowded(17, np.array(temps), indeces_file, visc_array)


In [ ]:
# fig = plt.figure(figsize=(2.1, 3), dpi=200) 
# plt.errorbar(np.array(temps), DT_corr, DT_err, fmt = 'o--')

# plt.ylabel('$D_t$ ($\\mathrm{\\AA}^2$/ns)')
# plt.xlabel('$T$ (K)')
# # plt.ylim((0, 1.7))
# plt.show()